In [20]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
import requests
from bs4 import BeautifulSoup as BS
import re
import pandas as pd

In [2]:
driver = webdriver.Chrome(ChromeDriverManager().install())



====== WebDriver manager ======
Current google-chrome version is 102.0.5005
Get LATEST chromedriver version for 102.0.5005 google-chrome
Driver [/Users/abigailasper/.wdm/drivers/chromedriver/mac64/102.0.5005.61/chromedriver] found in cache
/var/folders/kz/f2x26zgj2q5_y3k46h56qkw80000gn/T/ipykernel_50218/1503906442.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())


## log in to GoodReads

In [75]:
driver.quit()
driver.get("https://www.goodreads.com/user/sign_in")

MaxRetryError: HTTPConnectionPool(host='localhost', port=49320): Max retries exceeded with url: /session/77aa59494937acc945e2d204a53bad80/url (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7f860cf20ac0>: Failed to establish a new connection: [Errno 61] Connection refused'))

In [4]:
#select "sign in with email" option
#https://www.geeksforgeeks.org/click-button-by-text-using-python-and-selenium/
button = driver.find_element_by_link_text("Sign in with email")
button.click()

/var/folders/kz/f2x26zgj2q5_y3k46h56qkw80000gn/T/ipykernel_50218/342847067.py:3: DeprecationWarning: find_element_by_link_text is deprecated. Please use find_element(by=By.LINK_TEXT, value=link_text) instead
  button = driver.find_element_by_link_text("Sign in with email")


In [5]:
username = driver.find_element(By.NAME, "email")
username.send_keys("abigailasper@gmail.com")

### import json & keys file to safely import GoodReads password

In [6]:
import json

In [7]:
with open('../keys/gr_pw.json') as fi:
    credentials = json.load(fi)

In [8]:
gr_pw = credentials['gr_pw']

In [9]:
password = driver.find_element(By.NAME, "password")
password.send_keys(gr_pw)

In [10]:
#submit login credentials
signin = driver.find_element(By.ID, "signInSubmit")
signin.click()

## navigate to desired shelf (Booktok Books) & begin scrape

In [11]:
driver.get("https://www.goodreads.com/shelf/show/booktok")

In [63]:
#https://www.geeksforgeeks.org/how-to-scrape-multiple-pages-using-selenium-in-python/
book_list = []

for page in range(1, 26, 1):
    
    page_url = "https://www.goodreads.com/shelf/show/booktok?page=" + str(page)
    driver.get(page_url)
    title = driver.find_elements(By.CLASS_NAME, "bookTitle")
    author = driver.find_elements(By.CLASS_NAME, "authorName")
    
    for i in range(len(title)):
        book_list.append([title[i].text, author[i].text])
  
print(book_list)
        

[['The Seven Husbands of Evelyn Hugo (Hardcover)', 'Taylor Jenkins Reid'], ['It Ends with Us (Kindle Edition)', 'Colleen Hoover'], ['The Song of Achilles (Paperback)', 'Madeline Miller'], ['The Invisible Life of Addie LaRue (Hardcover)', 'V.E. Schwab'], ['The Cruel Prince (The Folk of the Air, #1)', 'Holly Black'], ['From Blood and Ash (Blood and Ash, #1)', 'Jennifer L. Armentrout'], ['Six of Crows (Six of Crows, #1)', 'Leigh Bardugo'], ['They Both Die at the End (Hardcover)', 'Adam Silvera'], ['A Court of Thorns and Roses (A Court of Thorns and Roses, #1)', 'Sarah J. Maas'], ['Red, White & Royal Blue (Paperback)', 'Casey McQuiston'], ['The Love Hypothesis (Paperback)', 'Ali Hazelwood'], ['Ugly Love (Kindle Edition)', 'Colleen Hoover'], ['We Were Liars (Kindle Edition)', 'E. Lockhart'], ['People We Meet on Vacation (Paperback)', 'Emily Henry'], ['The Spanish Love Deception (ebook)', 'Elena Armas'], ['The Hating Game (ebook)', 'Sally Thorne'], ['Beach Read (Paperback)', 'Emily Henry'], 

In [64]:
#make list a dataframe
books_df = pd.DataFrame(book_list)

books_df.head()

,0,1
0,The Seven Husbands of Evelyn Hugo (Hardcover),Taylor Jenkins Reid
1,It Ends with Us (Kindle Edition),Colleen Hoover
2,The Song of Achilles (Paperback),Madeline Miller
3,The Invisible Life of Addie LaRue (Hardcover),V.E. Schwab
4,"The Cruel Prince (The Folk of the Air, #1)",Holly Black


In [72]:
#get urls

#urls = []

for page in range(1, 26, 1):
    
    page_url = "https://www.goodreads.com/shelf/show/booktok?page=" + str(page)
    driver.get(page_url)
    
    #https://stackoverflow.com/questions/29078072/how-to-get-link-value-inside-href
    #https://selenium-python.readthedocs.io/locating-elements.html
    hrefs = driver.find_elements(By.XPATH, "//a[@class='bookTitle']")
  
    for my_href in hrefs:
        urls.append(my_href.get_attribute("href"))
        
urls

MaxRetryError: HTTPConnectionPool(host='localhost', port=49320): Max retries exceeded with url: /session/77aa59494937acc945e2d204a53bad80/url (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7f860cdd55e0>: Failed to establish a new connection: [Errno 61] Connection refused'))

In [69]:
#rename columns
books_df.columns = ['title', 'author']

In [70]:
books_df.head(20)

,title,author
0,The Seven Husbands of Evelyn Hugo (Hardcover),Taylor Jenkins Reid
1,It Ends with Us (Kindle Edition),Colleen Hoover
2,The Song of Achilles (Paperback),Madeline Miller
3,The Invisible Life of Addie LaRue (Hardcover),V.E. Schwab
4,"The Cruel Prince (The Folk of the Air, #1)",Holly Black
5,"From Blood and Ash (Blood and Ash, #1)",Jennifer L. Armentrout
6,"Six of Crows (Six of Crows, #1)",Leigh Bardugo
7,They Both Die at the End (Hardcover),Adam Silvera
8,A Court of Thorns and Roses (A Court of Thorns...,Sarah J. Maas
9,"Red, White & Royal Blue (Paperback)",Casey McQuiston


In [71]:
#save dataframe as csv
books_df.to_csv('booktok_books.csv')